In [1]:
import urllib.request as urllib2
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import re
from unidecode import unidecode
import pickle

#import keras 
from keras.models import Sequential
from keras.layers import LSTM, Dense, Activation
from keras.optimizers import RMSprop
import h5py

Using TensorFlow backend.


In [4]:
# load all songs
songs = pd.read_csv('files/eminem_lyrics.csv', sep=';')

text =""

# merge all the lyrics together into one huge string
for index, row in songs['lyrics'].iteritems():
    text = text + str(row).lower()
    
# find all the unique chracters
chars = sorted(list(set(text)))
print('total chars:', len(chars))

# create a dictionary mapping chracter-to-index
char_indices = dict((c, i) for i, c in enumerate(chars))

# create a dictionary mapping index-to-chracter
indices_char = dict((i, c) for i, c in enumerate(chars))

# cut the text into sequences
maxlen = 20
step = 1 # step size at every iteration
sentences = [] # list of sequences
next_chars = [] # list of next chracters that our model should predict

# iterate over text and save sequences into lists
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
    
# create empty matrices for input and output sets 
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)

# iterate over the matrices and convert all characters to numbers
# basically Label Encoding process and One Hot vectorization
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

total chars: 56


In [5]:
# create sequential network, because we are passing activations
# down the network
model = Sequential()

# add LSTM layer
model.add(LSTM(512, input_shape=(maxlen, len(chars)), return_sequences=True))
model.add(LSTM(128))

# add Softmax layer to output one character 
model.add(Dense(len(chars)))
model.add(Activation('softmax'))

# compile the model and pick the loss and optimizer
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(lr=0.01))

# train the model
model.fit(x, y, batch_size=128, epochs=30)

Epoch 1/30
 17152/240956 [=>............................] - ETA: 14:07 - loss: 3.1069

KeyboardInterrupt: 

In [4]:
# save the model
model.save('files/Aiminem_model.h5')  # creates a HDF5 file 'my_model.h5'

In [5]:
objects_vars = {"chars": chars, "char_indices": char_indices, "indices_char": indices_char}


for name, object_var in objects_vars.items():    
    # save
    name_file = "files/" + name + ".pkl"
    outfile = open(name_file,'wb')
    pickle.dump(object_var, outfile)
    outfile.close()